# Sports Game Statistics

This file performs "wrangling" of sporting event data, combining farebox, gatecount, and game schedule data for all three teams. Later files may consume that data as needed.

## Setup

In [1]:
# Libraries.
import datetime as dt
import numpy as np
import os
import pandas as pd

# Feature libraries.
from features import date

In [2]:
# Other settings.

# Stations near TD Garden for analysis. Just use names (not locationid) as a surrogate key for station.
td_stations = ['North Station']

# Stations near Fenway for analysis. Fenway station itself is not in the gatecount dataset.
# Just use names (not locationid) as a surrogate key for station name.
fenway_stations = ['Kenmore Square', 'Hynes']

# Base units of timedeltas.
timedelta_base = 3.6e12

# List of holidays and other bad days to exclude from analysis.
bad_dates = '2013-01-01,2013-01-02,2013-01-21,2013-02-10,2013-03-08,2013-09-01,2013-09-02,2013-10-14,2013-11-11,2013-11-27,2013-11-28,2013-11-29,2013-12-24,2013-12-25,2013-12-26,2013-12-27,2013-12-28,2013-12-31,2014-01-01,2014-01-02,2014-01-03,2014-01-20,2014-09-01,2014-10-13,2014-11-11,2014-11-27,2014-11-28,2014-12-23,2014-12-24,2014-12-25,2014-12-26,2014-12-27,2014-12-28,2014-12-29,2014-12-31,2015-01-01,2015-01-02,2015-01-03,2015-01-27,2015-02-10,2015-02-11'
bad_dates = [dt.date(int(h[:4]),int(h[5:7]),int(h[-2:])) for h in bad_dates.split(',')]

# Bruins/Celtics season information.
bc_season_start_month = 10
bc_season_end_month   = 6
bc_season_months = np.array(range(bc_season_start_month,13)+range(1,bc_season_end_month+1))

# Baseball season information.
# There are little edge cases regarding the playoffs/WS and such. Just use April through October to make things easiest.
sox_season_start_month = 4
sox_season_end_month   = 10
sox_season_months = np.arange(sox_season_start_month, sox_season_end_month+1)

---
## Game Data

### Red Sox

In [3]:
# Load data.
sox = pd.read_csv('../../../data/sox_master.csv')

In [4]:
# Split date.
sox['date'] = sox.START_DATE.str.slice(0,10)
# Make opponent lower case.
sox.rename(columns={'OPPONENT':'opponent'}, inplace=True)
# Add zero playoff, late columns.
sox['playoff'] = 0
sox['late'] = 0
# Get day of week.
sox['day_of_week'] = pd.DatetimeIndex(sox.date).weekday

In [5]:
# Handle times.

# Hour.
sox['hour'] = sox.START_TIME.str.slice(0,2).astype(np.int)
# Minute.
sox['minute'] = sox.START_TIME.str.slice(3,5).astype(np.int)

# Add PM hours.
sox.ix[sox.START_TIME.str.contains(' PM'),'hour'] += 12

# Round minutes.
sox.minute = (sox.minute / 15).astype(np.int) * 15

# Re-derive time.
sox['time'] = sox['hour'].astype(np.str).str.pad(2, fillchar='0') + ':' + sox['minute'].astype(np.str).str.pad(2, fillchar='0') + ':00'

In [6]:
# Clarify playoffs. Only needed for 2013.
sox.ix[(pd.to_datetime(sox.date) >= dt.datetime(2013,10,4)) & (pd.to_datetime(sox.date) < dt.datetime(2014,1,1)),'playoff'] = 1

In [7]:
# Clarify late games.
sox.loc[sox.hour >= 18,'late'] = 1

In [8]:
# Remove any games with bad dates that must be excluded.
sox = sox[pd.to_datetime(sox.date).isin(bad_dates) == False]
# Also get rid of 2012 games.
sox = sox[pd.to_datetime(sox.date) >= dt.datetime(2013,1,1)]
# Clean up index.
sox.reset_index(drop=True, inplace=True)

In [9]:
# Create "game ID" based on index.
sox.sort('date', ascending=True, inplace=True)
sox.reset_index(drop=False, inplace=True)
sox.rename(columns={'index':'game_id'}, inplace=True)

In [10]:
# Add season column.
sox['season'] = pd.DatetimeIndex(sox.date).year

In [11]:
# Clean and reorder.
# sox.drop(['Unnamed: 0','START_DATE','START_TIME'], axis=1, inplace=True)
sox = sox[['game_id','season','date','day_of_week','time','hour','minute','late','opponent','playoff']]

In [12]:
sox.head()

,game_id,season,date,day_of_week,time,hour,minute,late,opponent,playoff
0,0,2013,2013-04-08,0,14:00:00,14,0,0,Baltimore,0
1,1,2013,2013-04-10,2,19:00:00,19,0,1,Baltimore,0
2,2,2013,2013-04-11,3,19:00:00,19,0,1,Baltimore,0
3,3,2013,2013-04-13,5,13:00:00,13,0,0,Tampa Bay,0
4,4,2013,2013-04-14,6,13:30:00,13,30,0,Tampa Bay,0


### Bruins

In [13]:
# Load data.
bruins = pd.read_csv('../../../data/bruins/home.csv')

In [14]:
# Lower case columns.
bruins.rename(columns={'Playoff':'playoff','Opponent':'opponent'}, inplace=True)

In [15]:
# Handle dates.
bruins['date'] = bruins.Datetime.str.slice(0,10)

# Get day of week.
bruins['day_of_week'] = pd.DatetimeIndex(bruins.date).weekday

# Hours.
bruins['hour'] = bruins.Datetime.str.slice(11,13).astype(np.int)

# Minutes.
bruins['minute'] = bruins.Datetime.str.slice(14,16).astype(np.int)

# General time.
bruins['time'] = bruins.Datetime.str.slice(11,16) + ':00'

# Handle late games.
bruins['late'] = 0
bruins.ix[bruins.hour >= 19,'late'] = 1

In [16]:
# Remove any games with bad dates that must be excluded.
bruins = bruins[pd.to_datetime(bruins.date).isin(bad_dates) == False]
# Don't need to worry about 2012 games because of the lockout.
# Remove anything after traffic time range.
bruins = bruins[pd.to_datetime(bruins.date) <= dt.datetime(2015,2,11)]

In [17]:
# Add season.
bruins['season'] = (pd.DatetimeIndex(bruins.date) - np.timedelta64(9,'M')).year

In [18]:
# Create "game ID" based on index.
bruins.reset_index(drop=False, inplace=True)
bruins.rename(columns={'index':'game_id'}, inplace=True)

In [19]:
# Clean and reorder.
# bruins.drop(['Home'], axis=1, inplace=True)
bruins = bruins[['game_id','season','date','day_of_week','time','hour','minute','late','opponent','playoff']]

In [20]:
bruins.head()

,game_id,season,date,day_of_week,time,hour,minute,late,opponent,playoff
0,0,2012,2013-01-19,5,19:00:00,19,0,1,New York Rangers,0
1,2,2012,2013-01-25,4,19:00:00,19,0,1,New York Islanders,0
2,3,2012,2013-01-29,1,19:00:00,19,0,1,New Jersey,0
3,4,2012,2013-01-31,3,19:00:00,19,0,1,Buffalo,0
4,5,2012,2013-02-12,1,19:30:00,19,30,1,New York Rangers,0


### Celtics

In [21]:
# Load data.
celtics = pd.read_csv('../../../data/celtics/home.csv')

In [22]:
# Lower case columns.
celtics.rename(columns={'Playoff':'playoff','Opponent':'opponent'}, inplace=True)

In [23]:
# Handle dates.
celtics['date'] = celtics.Datetime.str.slice(0,10)

# Add day of week.
celtics['day_of_week'] = pd.DatetimeIndex(celtics.date).weekday

# Hours.
celtics['hour'] = celtics.Datetime.str.slice(11,13).astype(np.int)

# Minutes.
celtics['minute'] = celtics.Datetime.str.slice(14,16).astype(np.int)

# General time.
celtics['time'] = celtics.Datetime.str.slice(11,16) + ':00'

# Handle late games.
celtics['late'] = 0
celtics.ix[celtics.hour >= 19,'late'] = 1

In [24]:
# Remove any games with bad dates that must be excluded.
celtics = celtics[pd.to_datetime(celtics.date).isin(bad_dates) == False]
# Also remove anything before 2013.
celtics = celtics[pd.to_datetime(celtics.date) >= dt.datetime(2013,1,1)]
# Remove anything after traffic time range.
celtics = celtics[pd.to_datetime(celtics.date) <= dt.datetime(2015,2,11)]
# Clean index.
celtics.reset_index(drop=True, inplace=True)

In [25]:
# Add season.
celtics['season'] = (pd.DatetimeIndex(celtics.date) - np.timedelta64(9,'M')).year

In [26]:
# Create "game ID" based on index.
celtics.reset_index(drop=False, inplace=True)
celtics.rename(columns={'index':'game_id'}, inplace=True)

In [27]:
# Clean and reorder.
# celtics.drop(['Home'], axis=1, inplace=True)
celtics = celtics[['game_id','season','date','day_of_week','time','hour','minute','late','opponent','playoff']]

In [28]:
celtics.head()

,game_id,season,date,day_of_week,time,hour,minute,late,opponent,playoff
0,0,2012,2013-01-04,4,20:00:00,20,0,1,Indiana,0
1,1,2012,2013-01-09,2,19:30:00,19,30,1,Phoenix,0
2,2,2012,2013-01-11,4,19:30:00,19,30,1,Houston,0
3,3,2012,2013-01-14,0,19:30:00,19,30,1,Charlotte,0
4,4,2012,2013-01-16,2,20:00:00,20,0,1,New Orleans,0


---
## Gatecount & Farebox Data

### Gatecount

In [29]:
# Import gatecount data.
def get_data():
    return pd.DataFrame.from_csv("../../../data/mbta.csv").reset_index()
gatecount_raw = date.init(get_data())

In [30]:
# Restrict to desired columns.
gatecount = gatecount_raw[['locationid','name','line_1','line_2','service_day','service_datetime','entries']].copy()

# Clean lines.
gatecount.loc[gatecount.line_2.isnull() == True,'line_2'] = 'None'

# Turn days/dates into datetimes.
gatecount.service_day = pd.to_datetime(gatecount.service_day)
gatecount.service_datetime = pd.to_datetime(gatecount.service_datetime)

# Get time of day.
gatecount['service_time'] = pd.DatetimeIndex(gatecount.service_datetime).time

# Add day of week.
gatecount = date.add_day_of_week(gatecount.copy())

# Remove bad days.
gatecount = gatecount[gatecount.service_day.isin(bad_dates) == False]

In [31]:
gatecount.head()

,locationid,name,line_1,line_2,service_day,service_datetime,entries,service_time,day_of_week
10293,1002,Andrew Square,Red,None,2013-01-03,2013-01-03 05:00:00,6,05:00:00,3
10294,1002,Andrew Square,Red,None,2013-01-03,2013-01-03 05:15:00,49,05:15:00,3
10295,1002,Andrew Square,Red,None,2013-01-03,2013-01-03 05:30:00,58,05:30:00,3
10296,1002,Andrew Square,Red,None,2013-01-03,2013-01-03 05:45:00,46,05:45:00,3
10297,1002,Andrew Square,Red,None,2013-01-03,2013-01-03 06:00:00,88,06:00:00,3


In [32]:
# Get station splits.
station_splits = pd.read_csv('../../../data/StationSplits.csv')
station_splits.fillna(0.0, inplace=True)
station_splits

,station,pct_blue,pct_green,pct_orange,pct_red,pct_silver
0,Park Street,0.00,0.58,0.00,0.42,0.00
1,Downtown Crossing,0.00,0.00,0.49,0.51,0.00
2,State Street,0.42,0.00,0.58,0.00,0.00
3,Government Center,0.10,0.90,0.00,0.00,0.00
4,Haymarket,0.00,0.49,0.51,0.00,0.00
5,North Station,0.00,0.34,0.66,0.00,0.00
6,South Station,0.00,0.00,0.00,0.89,0.11


### Farebox

In [33]:
# Path for routes.
routes_file = '../../../data/SignCodesAndRoutes.csv'

# Get routes.
routes = pd.read_csv(routes_file)
# Restrict to Green Line BCDE.
gl_routes = routes[(routes['mode'] == 'G/Line') & routes.publicroute.isin(['Green Line '+l for l in 'BCDE'])]
# Get signcodes.
gl_signcodeids = set(gl_routes.signcodeid)

# Get simplified version for merge.
gl_sc_pr = gl_routes[['signcodeid','publicroute']]

In [34]:
# Farebox directories.
farebox_root = '../../../data/farebox_all/'

# Empty container for data.
farebox_raw = None
# Traverse each file and add to the main.
for farebox_file in os.listdir(farebox_root):
    farebox_month = pd.read_csv(farebox_root+farebox_file)
    if farebox_raw is None:
        farebox_raw = farebox_month
    else:
        farebox_raw = farebox_raw.append(farebox_month)

In [35]:
# Restrict to green line. Discard old farebox data.
farebox = farebox_raw[farebox_raw.signcodeid.isin(gl_signcodeids)].copy()

# Calculate entries.
farebox['entries'] = farebox.charlie + farebox.cash

# Drop old columns.
farebox.drop(['charlie','cash'], axis=1, inplace=True)

# Rename some columns for consistency.
farebox.rename(columns={'servicedate':'service_day', 'servicetime':'service_time'}, inplace=True)

# Make service_day a string for now.
farebox.service_day = farebox.service_day.astype(np.str)

# Fix dates and times for post-midnight 
farebox_post_midnight = farebox[farebox.service_time >= 2400]
# Fix times.
farebox_post_midnight.service_time = farebox_post_midnight.service_time-2400
# Fix dates.
farebox_post_midnight.service_day = pd.to_datetime(farebox_post_midnight.service_day) + np.timedelta64(1,'D')
farebox_post_midnight.service_day = pd.DatetimeIndex(farebox_post_midnight.service_day).format()
# Get pre-midnight data.
farebox_pre_midnight = farebox[farebox.service_time < 2400]
# Recombine.
farebox = farebox_pre_midnight.append(farebox_post_midnight)
# Housekeeping.
del farebox_pre_midnight, farebox_post_midnight

# Zero-pad service times.
farebox.service_time = farebox.service_time.map('{:04}'.format)

# Derive datetime.
farebox['service_datetime'] = pd.to_datetime(farebox.service_day + ' ' + farebox.service_time)

# Re-derive service time.
farebox.service_time = pd.DatetimeIndex(farebox['service_datetime']).time

# Turn plain day into a datetime and get day of week.
farebox.service_day = pd.to_datetime(farebox.service_day)
farebox['day_of_week'] = pd.DatetimeIndex(farebox.service_day).weekday

# Identify outbound/outbound routes.
farebox['inbound'] = (farebox.signcodeid % 10).astype(np.bool)

# Merge with publicroute information.
farebox = farebox.merge(gl_sc_pr, on='signcodeid')

# Aggregate by inbound/outbound, publicroute.
farebox = farebox.groupby(['service_day','service_time','service_datetime','day_of_week','inbound','publicroute']).agg(np.sum).reset_index()

# Perform final aggregation in case of multiple signcodeid entries.
# gl_farebox_agg = gl_farebox.groupby(['service_day','service_time','service_datetime','day_of_week','inbound']).agg(np.sum).reset_index()
farebox.drop(['signcodeid'], axis=1, inplace=True)

# Remove bad dates.
farebox = farebox[farebox.service_day.isin(bad_dates) == False]

/usr/local/Cellar/python/2.7.6/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/core/generic.py:2005: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [36]:
farebox.head()

,service_day,service_time,service_datetime,day_of_week,inbound,publicroute,entries
1187,2013-01-03,00:00:00,2013-01-03,3,False,Green Line B,8
1188,2013-01-03,00:00:00,2013-01-03,3,False,Green Line C,6
1189,2013-01-03,00:00:00,2013-01-03,3,False,Green Line D,5
1190,2013-01-03,00:00:00,2013-01-03,3,True,Green Line B,8
1191,2013-01-03,00:00:00,2013-01-03,3,True,Green Line C,5


### Aggregate

#### All Games

It will be useful to have information about all games for purposes of calculating gameless days.

In [37]:
# Get all games.
all_games = bruins.append(celtics).append(sox)
# Get set of all game dates.
all_game_dates = all_games.date.unique()

In [38]:
all_game_dates[:10]

array(['2013-01-19', '2013-01-25', '2013-01-29', '2013-01-31',
       '2013-02-12', '2013-02-28', '2013-03-02', '2013-03-03',
       '2013-03-07', '2013-03-09'], dtype=object)

#### Gatecount Pregame

We need to allocate traffic by day-time-station-line.

In [39]:
# Copy gatecount.
gatecount_station_line = gatecount.copy()
# Remove NaNs.
gatecount_station_line = gatecount_station_line[gatecount_station_line.name.isnull() == False]

# Calculate entries by line. For the first pass, split evenly between lines for stations on 2 lines.
line_cols = []
for line in gatecount_station_line.line_1.unique():
    # Derive columns.
    line_col = 'entries_'+line.lower()
    line_cols.append(line_col)
    # Add entries.
    gatecount_station_line.ix[gatecount_station_line.line_1 == line,line_col] = gatecount_station_line.ix[gatecount_station_line.line_1 == line,'entries']
    gatecount_station_line.ix[gatecount_station_line.line_2 == line,line_col] = gatecount_station_line.ix[gatecount_station_line.line_2 == line,'entries']
    # Handle splits.
    gatecount_station_line.ix[gatecount_station_line.line_2 != 'None',line_col] = gatecount_station_line.ix[gatecount_station_line.line_2 != 'None',line_col]/2.
    # Clean NAs.
    gatecount_station_line.ix[gatecount_station_line[line_col].isnull() == True,line_col] = 0.

# Drop columns we don't need.
gatecount_station_line.drop(['locationid','line_1','line_2'], axis=1, inplace=True)

In [40]:
# Now handle splits where data is available.
# Traverse stations.
for i in xrange(len(station_splits)):
    station = station_splits.ix[i,'station']
    # Traverse cols.
    for pct in station_splits.columns[1:]:
        # Update entries.
        gatecount_station_line.ix[gatecount_station_line.name == station,pct.replace('pct','entries')] = gatecount_station_line.ix[gatecount_station_line.name == station,'entries']*station_splits.ix[i,pct]

In [41]:
gatecount_station_line[gatecount_station_line.name == 'South Station'].head()

,name,service_day,service_datetime,entries,service_time,day_of_week,entries_red,entries_blue,entries_orange,entries_green,entries_silver
10700,South Station,2013-01-03,2013-01-03 04:45:00,3,04:45:00,3,2.67,0,0,0,0.33
10701,South Station,2013-01-03,2013-01-03 05:00:00,5,05:00:00,3,4.45,0,0,0,0.55
10702,South Station,2013-01-03,2013-01-03 05:15:00,31,05:15:00,3,27.59,0,0,0,3.41
10703,South Station,2013-01-03,2013-01-03 05:30:00,33,05:30:00,3,29.37,0,0,0,3.63
10704,South Station,2013-01-03,2013-01-03 05:45:00,15,05:45:00,3,13.35,0,0,0,1.65


In [42]:
# Calculate gameless pregame ridership.
gatecount_station_line_gameless = gatecount_station_line[gatecount_station_line.service_day.isin(all_game_dates) == False]

#### Farebox Pregame

Similar idea to above. Aggregate by letter and by green line generally.

In [43]:
# Copy farebox.
fb_all = farebox.copy()

# Get aggregates by DOW, time.
fb_day_time = fb_all.groupby(['service_day','day_of_week','service_time','service_datetime']).agg({'entries':np.sum}).reset_index()
fb_day_time.rename(columns={'entries':'entries_green'}, inplace=True)

# TODO: get by Green Line route as well. Not important now—the above is fine for general aggregations.

In [44]:
fb_day_time.head()

,service_day,day_of_week,service_time,service_datetime,entries_green
0,2013-01-03,3,00:00:00,2013-01-03 00:00:00,48
1,2013-01-03,3,00:15:00,2013-01-03 00:15:00,31
2,2013-01-03,3,00:30:00,2013-01-03 00:30:00,29
3,2013-01-03,3,00:45:00,2013-01-03 00:45:00,1
4,2013-01-03,3,01:15:00,2013-01-03 01:15:00,2


In [45]:
# Calculate gameless pregame ridership.
fb_day_time_gameless = fb_day_time[fb_day_time.service_day.isin(all_game_dates) == False]

#### Gatecount Postgame

In [46]:
# Get records specific to our stations of interest.
gatecount_game_stations = gatecount[gatecount.name.isin(fenway_stations+td_stations)].copy()

# Set up station group.
gatecount_game_stations['team'] = 'sox'
gatecount_game_stations.ix[gatecount_game_stations.name.isin(td_stations),'team'] = 'bc'

# Restrict columns.
gatecount_game_stations = gatecount_game_stations[['team','service_day','day_of_week','service_time','service_datetime','entries']]

# Aggregate by team, day, time. Solves duplicate Hynes/Kenmore records.
gatecount_game_stations = gatecount_game_stations.groupby(['team','service_day','day_of_week','service_time','service_datetime']).sum().reset_index(drop=False)

In [47]:
gatecount_game_stations.head()

,team,service_day,day_of_week,service_time,service_datetime,entries
0,bc,2013-01-03,3,00:00:00,2013-01-04 00:00:00,14
1,bc,2013-01-03,3,00:15:00,2013-01-04 00:15:00,7
2,bc,2013-01-03,3,00:30:00,2013-01-04 00:30:00,3
3,bc,2013-01-03,3,00:45:00,2013-01-04 00:45:00,2
4,bc,2013-01-03,3,01:00:00,2013-01-04 01:00:00,0


---
## Join & Calculate Differences

### Pregame

In [48]:
# Get information about months.
gc_gameless_months = pd.DatetimeIndex(gatecount_station_line_gameless.service_day).month
fb_gameless_months = pd.DatetimeIndex(fb_day_time_gameless.service_day).month

# Traverse schedules.
for games, team in zip([bruins,celtics,sox],['bruins','celtics','sox']):
    games = games.copy()
    # Column modifications.
    games['game_time_obj'] = pd.to_datetime(games.date + ' ' + games.time)
    games.date = pd.to_datetime(games.date)
    games.rename(columns={'time':'game_time'}, inplace=True)
    
    ### Gatecount.
    
    # Get gameless data in this season and aggregate.
    if team == 'sox':
        season_pregame_gc_gameless = gatecount_station_line_gameless[
            (gc_gameless_months <= sox_season_end_month) &
            (gc_gameless_months >= sox_season_start_month)
        ]
    else:
        season_pregame_gc_gameless = gatecount_station_line_gameless[
            (gc_gameless_months <= bc_season_end_month) |
            (gc_gameless_months >= bc_season_start_month)
        ]
    # Aggregate. Get means of all lines by day of week and time of day.
    agg_cols = dict((c,np.mean) for c in season_pregame_gc_gameless.columns if c.startswith('entries_'))
    season_pregame_gc_gameless = season_pregame_gc_gameless.groupby(['name','day_of_week','service_time']).agg(agg_cols).reset_index()
    # Rename columns. Clarify mean entries.
    col_renames = dict((c,c.replace('entries','mean_entries')) for c in season_pregame_gc_gameless.columns if c.startswith('entries_'))
    season_pregame_gc_gameless.rename(columns=col_renames, inplace=True)
    # We now have our mean entries reference for gameless days by station.

    # Merge with game data.
    game_gc = gatecount_station_line.merge(games, left_on=['service_day'], right_on=['date'])
    # Calculate time delta and filter based on it.
    game_gc['time_until_game_start'] = game_gc.game_time_obj - game_gc.service_datetime
    game_gc['hours_until_game_start'] = game_gc.time_until_game_start.astype(np.int) / timedelta_base
    game_gc = game_gc[(game_gc.time_until_game_start >= np.timedelta64(-30,'m')) & (game_gc.time_until_game_start <= np.timedelta64(6,'h'))]
    # Prune columns.
    entry_cols = [c for c in game_gc.columns if c.startswith('entries_')]
    game_gc = game_gc[['name','day_of_week_x','service_time','game_id','hours_until_game_start'] + entry_cols]
    game_gc.rename(columns={'day_of_week_x':'day_of_week'}, inplace=True)
    # Join to gameless data.
    game_gc_vs_gameless = game_gc.merge(season_pregame_gc_gameless, on=['name','day_of_week','service_time'])
    # Calculate lift.
    for c in entry_cols:
        game_gc_vs_gameless['lift_'+c] = game_gc_vs_gameless[c]-game_gc_vs_gameless['mean_'+c]
    # TODO: update lift for split stations.
    # Sum up lift per game.
    lift_cols = [c for c in game_gc_vs_gameless.columns if c.startswith('lift_entries_')]
    agg_sum_cols = dict((c,np.sum) for c in lift_cols)
    game_gc_vs_gameless = game_gc_vs_gameless.groupby(['game_id','day_of_week','hours_until_game_start']).agg(agg_sum_cols).reset_index().sort(['game_id','hours_until_game_start'])
    # Now get hourly averages.
    agg_mean_cols = dict((c,np.mean) for c in lift_cols)
    game_gc_vs_gameless_mean = game_gc_vs_gameless.groupby('hours_until_game_start').agg(agg_mean_cols).reset_index()
    # Drop silver.
    game_gc_vs_gameless_mean.drop('lift_entries_silver', axis=1, inplace=True)
    
    ### Farebox.
    
    # Get gameless data in this season and aggregate.
    if team == 'sox':
        season_pregame_fb_gameless = fb_day_time_gameless[
            (fb_gameless_months <= sox_season_end_month) &
            (fb_gameless_months >= sox_season_start_month)
        ]
    else:
        season_pregame_fb_gameless = fb_day_time_gameless[
            (fb_gameless_months <= bc_season_end_month) |
            (fb_gameless_months >= bc_season_start_month)
        ]
    # Aggregate. Get means by day of week and time of day.
    season_pregame_fb_gameless = season_pregame_fb_gameless.groupby(['day_of_week','service_time']).agg({'entries_green':np.mean}).reset_index()
    # Rename columns. Clarify mean entries.
    season_pregame_fb_gameless.rename(columns={'entries_green':'mean_entries_green'}, inplace=True)
    # We now have our mean entries reference for gameless days for Green Line Farebox data.

    # Merge with game data.
    game_fb = fb_day_time.merge(games, left_on=['service_day'], right_on=['date'])
    # Calculate time delta and filter based on it.
    game_fb['time_until_game_start'] = game_fb.game_time_obj - game_fb.service_datetime
    game_fb['hours_until_game_start'] = game_fb.time_until_game_start.astype(np.int) / timedelta_base
    game_fb = game_fb[(game_fb.time_until_game_start >= np.timedelta64(-30,'m')) & (game_fb.time_until_game_start <= np.timedelta64(6,'h'))]
    # Prune columns.
    entry_cols = [c for c in game_gc.columns if c.startswith('entries_')]
    game_fb = game_fb[['day_of_week_x','service_time','game_id','hours_until_game_start','entries_green']]
    game_fb.rename(columns={'day_of_week_x':'day_of_week'}, inplace=True)
    # Join to gameless data.
    game_fb_vs_gameless = game_fb.merge(season_pregame_fb_gameless, on=['day_of_week','service_time'])
    # Calculate lift. This will give us gameday-time lift.
    game_fb_vs_gameless['lift_entries_green'] = game_fb_vs_gameless['entries_green']-game_fb_vs_gameless['mean_entries_green']
    # Drop columns we don't need.
    game_fb_vs_gameless.drop(['mean_entries_green','entries_green','service_time'], axis=1, inplace=True)
    # Now get hourly averages.
    game_fb_vs_gameless_mean = game_fb_vs_gameless.groupby('hours_until_game_start').agg({'lift_entries_green':np.mean}).reset_index()
    
    ### Combined.

    # Simply append and sum.
    agg_lift = game_fb_vs_gameless.append(game_gc_vs_gameless).fillna(0)
    agg_lift = agg_lift.groupby(['game_id','day_of_week','hours_until_game_start']).sum().reset_index()
    
    # Trim columns for 171 verison.
    agg_lift_trunc = agg_lift.drop(['day_of_week','lift_entries_silver'],axis=1)
    
    # Assign variable depending on team.
    if team == 'bruins':
        bruins_pregame = agg_lift_trunc.copy()
    if team == 'celtics':
        celtics_pregame = agg_lift_trunc.copy()
    if team == 'sox':
        sox_pregame = agg_lift_trunc.copy()

### Postgame

In [49]:
# Get information about months.
gc_months = pd.DatetimeIndex(gatecount_game_stations.service_day).month

# Traverse schedules.
for games, team in zip([bruins,celtics,sox],['bruins','celtics','sox']):
    games = games.copy()
    # Column modifications.
    games['game_time_obj'] = pd.to_datetime(games.date + ' ' + games.time)
    games.date = pd.to_datetime(games.date)
    games.rename(columns={'time':'game_time'}, inplace=True)
    # Get appropriate postgame information.
    # Restrict to particular stations based on team, and only include seasonal data.
    if team == 'sox':
        season_ridership = gatecount_game_stations[
            (gatecount_game_stations.team == 'sox')   &
            (gc_months <= sox_season_end_month)  &
            (gc_months >= sox_season_start_month)
        ]
    else:
        season_ridership = gatecount_game_stations[
            (gatecount_game_stations.team == 'bc') &
            ((gc_months <= bc_season_end_month) | (gc_months >= bc_season_start_month))
        ]
    
    # Restrict to gameless days and aggregate.
    gameless_ridership = season_ridership[season_ridership.service_day.isin(all_game_dates) == False]
    gameless_ridership_agg = gameless_ridership.groupby(['day_of_week','service_time']).agg({'entries':np.mean}).reset_index()
    gameless_ridership_agg.rename(columns={'entries':'mean_entries'}, inplace=True)
    # Join ridership to games and restrict columns.
    game_ridership = games.merge(season_ridership, left_on='date', right_on='service_day')
    game_ridership = game_ridership[['game_id','day_of_week_x','service_time','service_datetime','game_time_obj','entries']]
    game_ridership.rename(columns={'day_of_week_x':'day_of_week'}, inplace=True)
    # Calculate time delta and filter based on it.
    game_ridership['time_since_game_start'] = game_ridership.service_datetime - game_ridership.game_time_obj
    game_ridership['hours_since_game_start'] = game_ridership.time_since_game_start.astype(np.int) / timedelta_base
    game_ridership = game_ridership[(game_ridership.time_since_game_start >= np.timedelta64(-30,'m')) & (game_ridership.time_since_game_start <= np.timedelta64(6,'h'))]
    # Join non-game means and calculate lift.
    game_ridership_vs_gameless = game_ridership.merge(gameless_ridership_agg, on=['day_of_week','service_time'])
    game_ridership_vs_gameless['entry_lift'] = game_ridership_vs_gameless.entries - game_ridership_vs_gameless.mean_entries
    
    # Trim columns and sort for 171 usage.
    game_ridership_vs_gameless_trunc = game_ridership_vs_gameless[['game_id','hours_since_game_start','entry_lift']].copy()
    game_ridership_vs_gameless_trunc.sort(['game_id','hours_since_game_start'], inplace=True)
    
    # Assign variable depending on team.
    if team == 'bruins':
        bruins_postgame = game_ridership_vs_gameless_trunc.copy()
    if team == 'celtics':
        celtics_postgame = game_ridership_vs_gameless_trunc.copy()
    if team == 'sox':
        sox_postgame = game_ridership_vs_gameless_trunc.copy()

---
## Output Data

Output data for analysis. 

In [53]:
# Write team schedules.
sox.to_csv('../../../data/sports_analysis/sox.csv', index=False)
bruins.to_csv('../../../data/sports_analysis/bruins.csv', index=False)
celtics.to_csv('../../../data/sports_analysis/celtics.csv', index=False)

In [54]:
# Save all pregame data.
bruins_pregame.to_csv('../../../data/sports_analysis/bruins_pregame.csv',   index=False)
celtics_pregame.to_csv('../../../data/sports_analysis/celtics_pregame.csv', index=False)
sox_pregame.to_csv('../../../data/sports_analysis/sox_pregame.csv',         index=False)

In [55]:
# Save all postgame data.
bruins_postgame.to_csv('../../../data/sports_analysis/bruins_postgame.csv',   index=False)
celtics_postgame.to_csv('../../../data/sports_analysis/celtics_postgame.csv', index=False)
sox_postgame.to_csv('../../../data/sports_analysis/sox_postgame.csv',         index=False)

---
## CS 171 Final Project Data

This data is also used in our CS 171 project. The following code simply saves data that has already been created above in a convenient local location.

In [359]:
# Write team schedules.
sox.to_csv('../../../../../CS 171/cs171-gameday/data/sox.csv', index=False)
bruins.to_csv('../../../../../CS 171/cs171-gameday/data/bruins.csv', index=False)
celtics.to_csv('../../../../../CS 171/cs171-gameday/data/celtics.csv', index=False)

In [493]:
# Save all pregame data.
bruins_pregame.to_csv('../../../../../CS 171/cs171-gameday/data/bruins_pregame.csv',   index=False)
celtics_pregame.to_csv('../../../../../CS 171/cs171-gameday/data/celtics_pregame.csv', index=False)
sox_pregame.to_csv('../../../../../CS 171/cs171-gameday/data/sox_pregame.csv',         index=False)

In [495]:
# Save all postgame data.
bruins_postgame.to_csv('../../../../../CS 171/cs171-gameday/data/bruins_postgame.csv',   index=False)
celtics_postgame.to_csv('../../../../../CS 171/cs171-gameday/data/celtics_postgame.csv', index=False)
sox_postgame.to_csv('../../../../../CS 171/cs171-gameday/data/sox_postgame.csv',         index=False)